In [152]:
# Importaciones y librerías
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,  mean_squared_error, classification_report
import numpy as np
import joblib
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV,  train_test_split, GridSearchCV
import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [153]:
# Cargar los datos
df = pd.read_csv('../../data/processed/JNJ_clean.csv')

# Crear las variables de entrenamiento y prueba, asegurándose de que los datos de entrenamiento sean hasta febrero de 2025
train_df = df[df['Date'] <= '2025-02-28']
test_df = df[df['Date'] >= '2025-03-01']

# Características (X) y objetivo (y)
X_train = train_df.drop(columns=['target', 'Close', 'Open', 'High','Low', 'Date'])
y_train = train_df['target']

X_test = test_df.drop(columns=['target', 'Close', 'Open', 'High','Low','Date'])
y_test = test_df['target']

In [154]:
# Función de entrenamiento y evaluación
def train_and_evaluate(model, X_train, X_test, y_train, y_test, model_name, is_classification=True):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    if is_classification:
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        
        print(f'{model_name} Accuracy: {accuracy:.4f}')
        print(f'{model_name} Precision (Macro): {precision:.4f}')
        print(f'{model_name} Recall (Macro): {recall:.4f}')
        print(f'Matriz de Confusión para {model_name}:\n{confusion_matrix(y_test, y_pred)}')
        print(f'Reporte de Clasificación para {model_name}:\n{classification_report(y_test, y_pred)}')
        
        # Importancia de características (si está disponible)
        if hasattr(model, 'feature_importances_'):
            feature_importances = model.feature_importances_
            # Se asume que X_train es un DataFrame para tener nombres de columnas
            feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'feature_{i}' for i in range(X_train.shape[1])]
            print(f'Importancia de Características para {model_name}:')
            for feature, importance in zip(feature_names, feature_importances):
                print(f'{feature}: {importance:.4f}')
        else:
            print(f'{model_name} no tiene importancias disponibles.')
    else:
        # Ejemplo para regresión (en caso de ser necesario)
        from sklearn.metrics import mean_squared_error
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        print(f'{model_name} RMSE: {rmse:.4f}')
        
    # Guardar el modelo entrenado
    joblib.dump(model, f'../../models/{model_name}_JNJ.pkl')
    return model

In [155]:
# Función de búsqueda de hiperparámetros y validación cruzada
def hyperparameter_tuning(model, X_train, y_train, param_grid, is_classification=True):
    # Se define la métrica a 'precision_macro' si es clasificación
    scoring = 'precision_macro' if is_classification else 'neg_mean_squared_error'
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = RandomizedSearchCV(model, 
                                     param_distributions=param_grid, 
                                     n_iter=10,  # Se aumentó para explorar más combinaciones
                                     cv=cv, 
                                     random_state=42, 
                                     n_jobs=-1, 
                                     scoring="precision")
    grid_search.fit(X_train, y_train)
    
    print(f"Mejores parámetros para {model.__class__.__name__}: {grid_search.best_params_}")
    return grid_search.best_estimator_

In [188]:
# Definición de modelos base
models = {
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(estimator=DecisionTreeClassifier(),random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LightGBM": lgb.LGBMClassifier(random_state=42, verbose=-1),
    "XGBoost": XGBClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(random_state=42, verbose=0)
}

# Rangos de hiperparámetros ampliados
param_grids = {
    "DecisionTree": {
        "criterion": ["gini", "entropy", "log_loss"],      # Tipos de función de impureza
        "max_depth": [3, 5, 10, 20, 30, None],              # Límite de profundidad (más opciones)
        "min_samples_split": [2, 5, 10, 20],                # Mínimo para dividir un nodo
        "min_samples_leaf": [1, 2, 5, 10],                  # Mínimo de muestras en hojas
        "max_features": [None, "sqrt", "log2"],             # Subconjunto de variables para división
        "class_weight": [None, "balanced"] 
    },
    "RandomForest": {
        "n_estimators": [200, 500, 800],
        "max_depth": [10, 20, 30, None],
        "max_features": ['sqrt', 'log2', None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "bootstrap": [True, False],
        "class_weight": [None, "balanced"]
    },
    "AdaBoost": {
        "n_estimators": [100, 200, 300, 400],        # Más iteraciones (mejor capacidad)
        "learning_rate": [0.01,0.05, 0.1, 0.3, 0.5, 1.0],  # Fino control del aprendizaje
        "estimator__max_depth": [1, 2, 3],           # Control de complejidad del árbol
        "estimator__min_samples_split": [2, 5, 10],  # Regularización
        "estimator__min_samples_leaf": [1, 2, 5],    # Más regularización
        "estimator__class_weight": [None, "balanced"]  # Mejor manejo del desbalance
    },
    "GradientBoosting": {
        "n_estimators": [100, 200, 300],         # Más árboles para mayor capacidad
        "learning_rate": [0.001, 0.05, 0.1, 0.2],       # Aprendizaje más rápido
        "max_depth": [4, 6, 8],                  # Mayor profundidad para captar interacciones
        "min_samples_split": [2, 5, 10],         # Regularización ligera (evita sobreajuste)
        "min_samples_leaf": [1, 3, 5],           # Controla hojas pequeñas que podrían sobreajustar
        "subsample": [0.8, 1.0],                 # Stochastic gradient boosting (para reducir overfitting)
        "max_features": ['sqrt', 'log2', None] 
    },
    "LightGBM": {
        "n_estimators": [50, 100, 200, 500],
        "learning_rate": [0.001, 0.01, 0.05, 0.1],
        "num_leaves": [31, 50, 70, 100],
        "min_child_samples": [5, 10, 20],
        "max_depth": [-1, 3, 5, 7, 10],
        "feature_fraction": [0.7, 0.8, 0.9, 1.0],
        "bagging_fraction": [0.7, 0.8, 0.9, 1.0],
        "bagging_freq": [0, 1, 5],
        "min_gain_to_split": [0, 0.01, 0.1],
        "lambda_l1": [0, 0.1, 1],
        "lambda_l2": [0, 0.1, 1]
    },
    "XGBoost": {
        "n_estimators": [300, 500],              # Capacidad razonable sin exagerar
        "learning_rate": [0.001,0.01, 0.05, 0.1],            # Eliminamos extremos poco útiles
        "max_depth": [3, 5, 7],                  # Suficiente para captar relaciones
        "subsample": [0.7, 1.0],                 # Aleatoriedad en filas
        "colsample_bytree": [0.7, 1.0],          # Aleatoriedad en columnas
        "scale_pos_weight": [1, 5]               # Control básico de desbalance
        },
    "CatBoost": {
        "iterations": [500, 1000, 1500],            # Mantener un rango equilibrado, 2000 puede ser demasiado si hay overfitting
        "learning_rate": [0.001,0.01, 0.05, 0.1],          # 0.001 puede ser muy lento, lo quitamos
        "depth": [4, 6, 8, 10],                      # OK
        "l2_leaf_reg": [1, 3, 5, 7, 9],              # Regularización L2 para evitar sobreajuste
        "bagging_temperature": [0, 0.5, 1.0],        # Controla la aleatoriedad del bootstrapping
        "random_strength": [0.5, 1, 2],              # Regularización del uso de características
        "border_count": [32, 64, 128],               # Cantidad de splits posibles en variables continuas
        "grow_policy": ['SymmetricTree', 'Depthwise', 'Lossguide'],  # Estructura de árboles
        "eval_metric": ['Accuracy', 'F1']           # Métricas de evaluación
    }
}

# Variables para almacenar el mejor modelo global y su score
best_model = None
best_score = -np.inf  # Se maximiza la métrica (precisión macro)

In [168]:
model_name = "DecisionTree"
model = models[model_name]
print(f"\nEntrenando {model_name}...")

# Ajuste de hiperparámetros usando validación cruzada para DecisionTree
model_tuned = hyperparameter_tuning(model, X_train, y_train, param_grids[model_name], is_classification=True)

# Entrenamiento y evaluación del modelo
trained_model = train_and_evaluate(model_tuned, X_train, X_test, y_train, y_test, model_name, is_classification=True)

# Evaluación basada en la precisión macro
score = precision_score(y_test, trained_model.predict(X_test), average='macro')
print(f"Precisión (Macro) en Test para {model_name}: {score:.4f}")

# Actualización del mejor modelo en función de la métrica
if score > best_score:
    best_score = score
    best_model = trained_model


Entrenando DecisionTree...
Mejores parámetros para DecisionTreeClassifier: {'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 3, 'criterion': 'log_loss', 'class_weight': 'balanced'}
DecisionTree Accuracy: 0.4667
DecisionTree Precision (Macro): 0.4750
DecisionTree Recall (Macro): 0.4777
Matriz de Confusión para DecisionTree:
[[ 9  5]
 [11  5]]
Reporte de Clasificación para DecisionTree:
              precision    recall  f1-score   support

           0       0.45      0.64      0.53        14
           1       0.50      0.31      0.38        16

    accuracy                           0.47        30
   macro avg       0.47      0.48      0.46        30
weighted avg       0.48      0.47      0.45        30

Importancia de Características para DecisionTree:
log_vol: 0.0000
year: 0.0000
month: 0.0000
day: 0.0000
day_of_week: 0.0000
is_month_end: 0.0000
price_diff: 0.0000
pct_diff: 0.0000
return_daily: 0.0000
return_lag_1: 0.0000
return_lag_2: 0.0000
ret

In [177]:
model_name = "RandomForest"
model = models[model_name]
print(f"\nEntrenando {model_name}...")

# Ajuste de hiperparámetros usando validación cruzada para RandomForest
model_tuned = hyperparameter_tuning(model, X_train, y_train, param_grids[model_name], is_classification=True)

# Entrenamiento y evaluación del modelo
trained_model = train_and_evaluate(model_tuned, X_train, X_test, y_train, y_test, model_name, is_classification=True)

# Evaluación basada en la precisión macro
score = precision_score(y_test, trained_model.predict(X_test), average='macro')
print(f"Precisión (Macro) en Test para {model_name}: {score:.4f}")

# Actualización del mejor modelo en función de la métrica
if score > best_score:
    best_score = score
    best_model = trained_model



Entrenando RandomForest...
Mejores parámetros para RandomForestClassifier: {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20, 'class_weight': None, 'bootstrap': True}
RandomForest Accuracy: 0.5333
RandomForest Precision (Macro): 0.5312
RandomForest Recall (Macro): 0.5312
Matriz de Confusión para RandomForest:
[[7 7]
 [7 9]]
Reporte de Clasificación para RandomForest:
              precision    recall  f1-score   support

           0       0.50      0.50      0.50        14
           1       0.56      0.56      0.56        16

    accuracy                           0.53        30
   macro avg       0.53      0.53      0.53        30
weighted avg       0.53      0.53      0.53        30

Importancia de Características para RandomForest:
log_vol: 0.0493
year: 0.0132
month: 0.0275
day: 0.0376
day_of_week: 0.0204
is_month_end: 0.0006
price_diff: 0.0396
pct_diff: 0.0405
return_daily: 0.0446
return_lag_1: 0.0438
return_lag_2: 0.046

In [173]:
model_name = "AdaBoost"
model = models[model_name]
print(f"\nEntrenando {model_name}...")

# Ajuste de hiperparámetros usando validación cruzada para AdaBoost
model_tuned = hyperparameter_tuning(model, X_train, y_train, param_grids[model_name], is_classification=True)

# Entrenamiento y evaluación del modelo
trained_model = train_and_evaluate(model_tuned, X_train, X_test, y_train, y_test, model_name, is_classification=True)

# Evaluación basada en la precisión macro
score = precision_score(y_test, trained_model.predict(X_test), average='macro')
print(f"Precisión (Macro) en Test para {model_name}: {score:.4f}")

# Actualización del mejor modelo en función de la métrica
if score > best_score:
    best_score = score
    best_model = trained_model



Entrenando AdaBoost...
Mejores parámetros para AdaBoostClassifier: {'n_estimators': 100, 'learning_rate': 1.0, 'estimator__min_samples_split': 10, 'estimator__min_samples_leaf': 5, 'estimator__max_depth': 1, 'estimator__class_weight': 'balanced'}
AdaBoost Accuracy: 0.4333
AdaBoost Precision (Macro): 0.4367
AdaBoost Recall (Macro): 0.4375
Matriz de Confusión para AdaBoost:
[[ 7  7]
 [10  6]]
Reporte de Clasificación para AdaBoost:
              precision    recall  f1-score   support

           0       0.41      0.50      0.45        14
           1       0.46      0.38      0.41        16

    accuracy                           0.43        30
   macro avg       0.44      0.44      0.43        30
weighted avg       0.44      0.43      0.43        30

Importancia de Características para AdaBoost:
log_vol: 0.0000
year: 0.0000
month: 0.0000
day: 0.0685
day_of_week: 0.0000
is_month_end: 0.0000
price_diff: 0.0000
pct_diff: 0.0000
return_daily: 0.0000
return_lag_1: 0.0000
return_lag_2: 0.00

In [166]:
model_name = "GradientBoosting"
model = models[model_name]
print(f"\nEntrenando {model_name}...")

# Ajuste de hiperparámetros usando validación cruzada para GradientBoosting
model_tuned = hyperparameter_tuning(model, X_train, y_train, param_grids[model_name], is_classification=True)

# Entrenamiento y evaluación del modelo
trained_model = train_and_evaluate(model_tuned, X_train, X_test, y_train, y_test, model_name, is_classification=True)

# Evaluación basada en la precisión macro
score = precision_score(y_test, trained_model.predict(X_test), average='macro')
print(f"Precisión (Macro) en Test para {model_name}: {score:.4f}")

# Actualización del mejor modelo en función de la métrica
if score > best_score:
    best_score = score
    best_model = trained_model



Entrenando GradientBoosting...
Mejores parámetros para GradientBoostingClassifier: {'subsample': 0.8, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 6, 'learning_rate': 0.2}
GradientBoosting Accuracy: 0.5000
GradientBoosting Precision (Macro): 0.5045
GradientBoosting Recall (Macro): 0.5045
Matriz de Confusión para GradientBoosting:
[[8 6]
 [9 7]]
Reporte de Clasificación para GradientBoosting:
              precision    recall  f1-score   support

           0       0.47      0.57      0.52        14
           1       0.54      0.44      0.48        16

    accuracy                           0.50        30
   macro avg       0.50      0.50      0.50        30
weighted avg       0.51      0.50      0.50        30

Importancia de Características para GradientBoosting:
log_vol: 0.0439
year: 0.0091
month: 0.0255
day: 0.0335
day_of_week: 0.0210
is_month_end: 0.0000
price_diff: 0.0452
pct_diff: 0.0271
return_daily: 0.0402
return_lag

In [161]:
model_name = "LightGBM"
model = models[model_name]
print(f"\nEntrenando {model_name}...")

# Ajuste de hiperparámetros usando validación cruzada para LightGBM
model_tuned = hyperparameter_tuning(model, X_train, y_train, param_grids[model_name], is_classification=True)

# Entrenamiento y evaluación del modelo
trained_model = train_and_evaluate(model_tuned, X_train, X_test, y_train, y_test, model_name, is_classification=True)

# Evaluación basada en la precisión macro
score = precision_score(y_test, trained_model.predict(X_test), average='macro')
print(f"Precisión (Macro) en Test para {model_name}: {score:.4f}")

# Actualización del mejor modelo en función de la métrica
if score > best_score:
    best_score = score
    best_model = trained_model



Entrenando LightGBM...
Mejores parámetros para LGBMClassifier: {'num_leaves': 70, 'n_estimators': 200, 'min_gain_to_split': 0.01, 'min_child_samples': 20, 'max_depth': 10, 'learning_rate': 0.1, 'lambda_l2': 1, 'lambda_l1': 1, 'feature_fraction': 0.8, 'bagging_freq': 5, 'bagging_fraction': 0.7}
LightGBM Accuracy: 0.6000
LightGBM Precision (Macro): 0.5982
LightGBM Recall (Macro): 0.5982
Matriz de Confusión para LightGBM:
[[ 8  6]
 [ 6 10]]
Reporte de Clasificación para LightGBM:
              precision    recall  f1-score   support

           0       0.57      0.57      0.57        14
           1       0.62      0.62      0.62        16

    accuracy                           0.60        30
   macro avg       0.60      0.60      0.60        30
weighted avg       0.60      0.60      0.60        30

Importancia de Características para LightGBM:
log_vol: 267.0000
year: 54.0000
month: 101.0000
day: 182.0000
day_of_week: 54.0000
is_month_end: 1.0000
price_diff: 162.0000
pct_diff: 120.0000


In [189]:
model_name = "XGBoost"
model = models[model_name]
print(f"\nEntrenando {model_name}...")

# Ajuste de hiperparámetros usando validación cruzada para XGBoost
model_tuned = hyperparameter_tuning(model, X_train, y_train, param_grids[model_name], is_classification=True)

# Entrenamiento y evaluación del modelo
trained_model = train_and_evaluate(model_tuned, X_train, X_test, y_train, y_test, model_name, is_classification=True)

# Evaluación basada en la precisión macro
score = precision_score(y_test, trained_model.predict(X_test), average='macro')
print(f"Precisión (Macro) en Test para {model_name}: {score:.4f}")

# Actualización del mejor modelo en función de la métrica
if score > best_score:
    best_score = score
    best_model = trained_model



Entrenando XGBoost...
Mejores parámetros para XGBClassifier: {'subsample': 0.7, 'scale_pos_weight': 1, 'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
XGBoost Accuracy: 0.6000
XGBoost Precision (Macro): 0.6027
XGBoost Recall (Macro): 0.6027
Matriz de Confusión para XGBoost:
[[9 5]
 [7 9]]
Reporte de Clasificación para XGBoost:
              precision    recall  f1-score   support

           0       0.56      0.64      0.60        14
           1       0.64      0.56      0.60        16

    accuracy                           0.60        30
   macro avg       0.60      0.60      0.60        30
weighted avg       0.61      0.60      0.60        30

Importancia de Características para XGBoost:
log_vol: 0.0375
year: 0.0314
month: 0.0346
day: 0.0337
day_of_week: 0.0351
is_month_end: 0.0428
price_diff: 0.0332
pct_diff: 0.0438
return_daily: 0.0367
return_lag_1: 0.0344
return_lag_2: 0.0366
return_lag_3: 0.0365
return_lag_4: 0.0389
return_lag_5: 0.0333
sma

In [186]:
model_name = "CatBoost"
model = models[model_name]
print(f"\nEntrenando {model_name}...")

# Ajuste de hiperparámetros usando validación cruzada para CatBoost
model_tuned = hyperparameter_tuning(model, X_train, y_train, param_grids[model_name], is_classification=True)

# Entrenamiento y evaluación del modelo
trained_model = train_and_evaluate(model_tuned, X_train, X_test, y_train, y_test, model_name, is_classification=True)

# Evaluación basada en la precisión macro
score = precision_score(y_test, trained_model.predict(X_test), average='macro')
print(f"Precisión (Macro) en Test para {model_name}: {score:.4f}")

# Actualización del mejor modelo en función de la métrica
if score > best_score:
    best_score = score
    best_model = trained_model



Entrenando CatBoost...
Mejores parámetros para CatBoostClassifier: {'random_strength': 2, 'learning_rate': 0.001, 'l2_leaf_reg': 1, 'iterations': 500, 'grow_policy': 'Lossguide', 'eval_metric': 'Accuracy', 'depth': 4, 'border_count': 128, 'bagging_temperature': 0.5}
CatBoost Accuracy: 0.6000
CatBoost Precision (Macro): 0.6250
CatBoost Recall (Macro): 0.5804
Matriz de Confusión para CatBoost:
[[ 4 10]
 [ 2 14]]
Reporte de Clasificación para CatBoost:
              precision    recall  f1-score   support

           0       0.67      0.29      0.40        14
           1       0.58      0.88      0.70        16

    accuracy                           0.60        30
   macro avg       0.62      0.58      0.55        30
weighted avg       0.62      0.60      0.56        30

Importancia de Características para CatBoost:
log_vol: 3.9804
year: 2.3454
month: 3.3097
day: 2.1614
day_of_week: 3.0106
is_month_end: 0.4375
price_diff: 2.4432
pct_diff: 2.0801
return_daily: 4.1629
return_lag_1: 3.072

In [187]:
print(f"\nMejor Modelo: {best_model} con precisión (macro) {best_score:.4f}")



Mejor Modelo: <catboost.core.CatBoostClassifier object at 0x317e9bbe0> con precisión (macro) 0.6250
